In [ ]:
"""
------------------------------------------------------------------------------
About
In this data analysis, the number of fast food stalls listed at IHK is calculated 
for each admin area. 
------------------------------------------------------------------------------
"""

In [ ]:
"""
------------------------------------------------------------------------------
Libraries
------------------------------------------------------------------------------
"""

import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt

from shapely.geometry import Point,Polygon, box

In [ ]:
"""
------------------------------------------------------------------------------
Import data
------------------------------------------------------------------------------
"""

df = pd.read_csv("../data/raw/IHKBerlin_Gewerbedaten.txt", dtype={"opendata_id": str, "planungsraum_id": str})

df.head(20)

In [ ]:
#subset
df_ihk = df[["latitude", "longitude","ihk_branch_id", "ihk_branch_desc", "nace_desc", "branch_top_level_desc", "business_type", "planungsraum_id","Prognoseraum", "Ortsteil"]]

df_ihk.head()


In [ ]:
#add column pgr_id
df_ihk["pgr_id"] = df_ihk["planungsraum_id"].str[:4]

df_ihk.head()

In [ ]:
#NAs
df_ihk.isna().sum()

In [ ]:
#drop relevant NAs
df_ihk.dropna(subset = "ihk_branch_desc", inplace=True)

In [ ]:
#unique values
df_ihk["business_type"].unique()

In [ ]:
df_ihk["branch_top_level_desc"].unique()

In [ ]:
df_ihk["nace_desc"].unique()

In [ ]:
#subset "Gastronomie"
df_gastro = df_ihk[df_ihk["branch_top_level_desc"] == "Gastronomie"]

df_gastro.head()

In [ ]:
#unique values
df_gastro["ihk_branch_desc"].unique()

In [ ]:
#category Imbissstuben u.ä.
df_imbiss = df_gastro[(df_gastro["ihk_branch_desc"] == "Imbissstuben u.ä.")]

df_imbiss.rename(columns={"Prognoseraum": "pgr_name"}, inplace=True)
df_imbiss.head()

In [ ]:
df_imbiss.shape

In [ ]:
df_imbiss.isna().sum()

In [ ]:
#row with NA
print(df_imbiss[df_imbiss.isna().any(axis=1)])


In [ ]:
#total number of Imbisstuben u.ä.
branchCount = df_imbiss.ihk_branch_desc.value_counts()


branchCount

In [ ]:
#check pgr_id
df_imbiss["pgr_id"].unique()

In [ ]:

#group by pgr_name and count rows
df_imbissCount = df_imbiss.groupby("pgr_name").size().reset_index(name="no_Imbiss")
#add first value pgr_id
df_imbissCount["pgr_id"] = df_imbiss.groupby("pgr_name")["pgr_id"].first().values

#subset
imbissCount= df_imbissCount[["pgr_id", "pgr_name", "no_Imbiss"]]

#sort descending
imbissCount.sort_values(by="no_Imbiss", ascending=False, inplace=True)
imbissCount.reset_index(drop=True, inplace=True)

imbissCount


In [ ]:
#csv
imbissCount.to_csv("../data/csv/fastFoodIHK.csv", index=False)

In [ ]:
"""
------------------------------------------------------------------------------
MAPS

GeoJSON and SVG files for the intersections per admin area
------------------------------------------------------------------------------
"""

In [ ]:
df_imbiss.head()

In [ ]:
"""
------------------------------------------------------------------------------
Geo data frame
------------------------------------------------------------------------------
"""

#point geometry from longitude and latitude
geometry = [Point(xy) for xy in zip(df_imbiss["longitude"], df_imbiss["latitude"])]
#geo data frame
gdf_imbiss = gpd.GeoDataFrame(df_imbiss, geometry=geometry)

In [ ]:
#crs
gdf_imbiss.set_crs(epsg=4326, inplace=True)

In [ ]:
#import data admin areas
admin_areas = gpd.read_file("../data/raw/Prognoseräume.json")

admin_areas.head()

In [ ]:
#check crs
admin_areas.crs

In [ ]:
"""
------------------------------------------------------------------------------
Checking invalid geometries
------------------------------------------------------------------------------
"""

#Check the combined dataframe for empty or invalid geometries
print(f"Empty geometry in gdf_imbiss: {len(gdf_imbiss[gdf_imbiss.is_empty])}")
print(f"Invalid in gdf_imbis: {len(gdf_imbiss[~gdf_imbiss.is_valid])}")

In [ ]:
"""
------------------------------------------------------------------------------
Functions
- point geometry to square
- fast food stalls per admin area, svg and geojson
------------------------------------------------------------------------------
"""
import math

def point_to_square(point, size=10):
    #create a square around the points with a defined length
    x, y = point.x, point.y
    half_size = size / 2
    return box(x - half_size, y - half_size, x + half_size, y + half_size)

def generate_hexagon(point, size=10):
    # Extract coordinates from the Point object
    center_x, center_y = point.x, point.y
    hexagon_points = []
    
    # Calculate the vertices of the hexagon
    for i in range(6):
        angle_deg = 60 * i
        angle_rad = math.radians(angle_deg)
        x = center_x + size * math.cos(angle_rad)
        y = center_y + size * math.sin(angle_rad)
        hexagon_points.append((x, y))
    
    # Return a Polygon created from the hexagon vertices
    return Polygon(hexagon_points)


def create_fastFoodGeom(admin_area, gdf_imbiss):

    admin_geometry = admin_area.geometry
    Prognoseraum = admin_area.get("pgr_name").replace(" ", "_").replace("/", "_").replace(";", "_")

    #check if geometry exists
    if admin_geometry is None or admin_geometry.is_empty:
        print(f"Invalid geometry for {Prognoseraum}")
        return
    
    #Find intersections between the Imbiss GeoDataFrame and the admin area geometry
    intersections_fastFood = gdf_imbiss[gdf_imbiss.intersects(admin_geometry)]

    #Geo Data Frame for the admin area
    gdf_admin = gpd.GeoDataFrame({"geometry": [admin_geometry]}, crs=gdf_imbiss.crs)

    #Convert Imbiss points to squares
    squares = intersections_fastFood.geometry.apply(lambda point: generate_hexagon(point, size=10))
    
    gdf_intersection_squares = gpd.GeoDataFrame(geometry=squares, crs=gdf_imbiss.crs)

    #Plot admin area
    fig, ax = plt.subplots(figsize=(30, 30))
    gdf_admin.plot(ax=ax, color="lightgrey")

    #Plot Imbiss points within the admin area
    if not gdf_intersection_squares.empty:
        gdf_intersection_squares.plot(ax=ax, color="red", alpha=0.7, edgecolor="black", linewidth=0.5)

    #add text
    centroid = admin_geometry.centroid
    ax.text(centroid.x, centroid.y, Prognoseraum, fontsize=12, ha="center", va="center")

    #convert squares back to EPSG:4326
    gdf_intersection_squares = gdf_intersection_squares.to_crs(epsg=4326)

    #save squares as geojson
    gdf_intersection_squares.to_file(f"../maps/prognoseraum/{Prognoseraum}/{Prognoseraum}_imbiss.geojson", driver="GeoJSON")

    #save fig as svg
    # fig.savefig(f"../maps/svg/{Prognoseraum}_imbiss.svg", format="svg")

    # plt.show()
    # plt.close(fig)

       


In [ ]:
#convert to ESPG:32633
admin_areas = admin_areas.to_crs(epsg=32633) 

if gdf_imbiss.crs != admin_areas.crs:
    gdf_imbiss = gdf_imbiss.to_crs(admin_areas.crs)

print(admin_areas.crs)
print(gdf_imbiss.crs)

In [ ]:
result = admin_areas.apply(
    lambda row: create_fastFoodGeom(row, gdf_imbiss), axis=1
)